In [71]:

import logging
logging.basicConfig()
logger = logging.getLogger('')
logger.setLevel(logging.INFO)

logger.debug("debug")

In [145]:
# %load sudoku.py
import pandas as pd
import numpy as np
def getPuzzle(puzale_str):
    return np.array(list(puzale_str)).reshape(9,9)

def isSolved(np_cells):
    list=np_cells.reshape(81)
    logger.debug(list)
    return not ('0' in list)

def map2orginal(row, col, block_row, block_col):
    return (block_row*3 +row, block_col*3 + col)

def getBlockIndex(row_index, col_index):
    r= row_index//3
    c = col_index//3
    return (r,c)

def getBlock(block_row, block_col, np_cells):
    return np_cells[block_row*3:block_row*3+3, block_col*3:block_col*3+3]

def getBlockList(block_row, block_col, np_cells):
    return getBlock(block_row, block_col, np_cells).reshape(9)

def getRow(row, np_cells):
    return np_cells[row]

def getCol(col, np_cells):
    return np_cells[:, col]

def calculate_possibilitie(row_index, col_index, np_cells):
    row=getRow(row_index, np_cells)
    col=getCol(col_index, np_cells)
    block_row, block_col = getBlockIndex(row_index, col_index)
    block=getBlockList(block_row, block_col, np_cells)
    possibilities={i for i in "123456789"}
    for i in row:
        if i in possibilities:
            possibilities.remove(i)
    for j in col:
        if j in possibilities:
            possibilities.remove(j)
    for k in block:
        if k in possibilities:
            possibilities.remove(k)
    return (row_index, col_index, possibilities)

def calculate_cells(np_cells):
    solved=True
    changed=True
    results=[]
    iterations=0
    logger.debug(np_cells)
    while changed:
        iterations+=1
        changed=False
        results=[]
        for i in range(9):
            for j in range(9):
                if np_cells[i,j] == '0' :
                    results.append(calculate_possibilitie(i,j,np_cells))
                    solved=False


        for x,y,s in results:
            if len(s)==1:
                np_cells[x,y]=next(iter(s))
                changed=True
        
        logger.debug("results: {}".format(results))
        logger.debug("iterations: {}".format(iterations) )
        logger.debug("change: {}".format(changed) )

    logger.debug(np_cells)
    return solved

def addCellToMissingsMap(key, cell, missings_map):
    if key in missings_map.keys():
        missings_map[key].append(cell)
    else:
        missings_map[key]=[cell]

def getMissingValeusAndUnknownCells(block):
    missing_values={i for i in "123456789"}
    unknown_cells=[]
    for i in range(3):
        for j in range(3):
            v=block[i,j]
            if v=='0':
                unknown_cells.append((i,j))
            else:
                missing_values.remove(v)
    logger.debug("unknown_cells: {}".format(unknown_cells) )
    logger.debug("missing_values".format(missing_values))
    return (missing_values, unknown_cells)

class Block:
    def __init__(self, r, c, np_cells) -> None:
        self.block=getBlock(r,c,np_cells)
        logger.debug(self.block)
        self.block_row, self.block_col=(r,c)
        self.np_cells=np_cells

        self.missings_map=self.getMissingsMap()
        
        logger.debug(self.missings_map)

    def getMissingsMap(self):
        missing_values, unknown_cells = getMissingValeusAndUnknownCells(self.block)
        logger.debug("missing_values: {}, unknown_cells: {}".format(missing_values, unknown_cells))
        
        missings_map={}

        for key in missing_values:
            for cell in unknown_cells:
                logger.debug("key {}, cell: {}".format(key, cell))
                if self.couldfit(key, cell):
                    logger.debug("add {} to {}".format(cell, key))
                    addCellToMissingsMap(key, cell, missings_map)
                else:
                    logger.debug("could not add {} to {}".format(cell, key))
        return missings_map


    def couldfit(self, key, cell):
        row_index, col_index = map2orginal(cell[0], cell[1],self.block_row, self.block_col)

        row=getRow(row_index, self.np_cells)
       
        col=getCol(col_index, self.np_cells)
       
        result =True
        if key in row:
            logger.debug("found {} in {}".format(key, row))
            result=False
        else:
            logger.debug("No {} in row {}: {}".format(key, row_index, row))
        if key in col:
            logger.debug("found {} in {}".format(key, col))
            result=False
        else:
            logger.debug("No {} in col {}: {}".format(key,col_index, col))
       
        return result

    # def addpos(self, m, u):
    #     if m in self.missings.keys():
    #         poss=self.missings[m]
    #         poss.append(u)
    #     else:
    #         poss=[u]
    #         self.missings[m]=poss

    def removepos(self,removes):
        deleted=[]
        for r in removes:
            for key in self.missings_map.keys():
                if r in self.missings_map[key]:
                    self.missings_map[key].remove(r)
                    if len(self.missings_map[key])==0:
                        deleted.append(key)
                    
        for d in deleted:
            self.missings_map.pop(d)


    def calculate(self):
        removes=[]
        changed=False
        repeat=True
        while repeat:
            repeat=False
            for key in self.missings_map.keys():
                if len(self.missings_map[key]) == 1:
                    px, py=self.missings_map[key][0]
                    logger.debug("Solved {} at {}".format(key,self.missings_map[key][0]))
                    self.block[px, py]=key
                    removes.append((px,py))
                    logger.debug(self.block)
                    changed=True
                    repeat=False
            self.removepos(removes)
        
        return changed
    
def calculate_blocks(np_cells):
    changed=False
    for block_row in range(3):
        for block_col in range(3):
            logger.debug("solving {},{}".format(block_row, block_col) )
            b=Block(block_row, block_col, np_cells)
            if b.calculate():
                changed=True
    return changed

def single_solve(np_cells):
    changed=True
    solved=False
    while changed:
        changed=False
        if calculate_cells(np_cells):
            solved=True
        else:
            if calculate_blocks(np_cells):
                changed=True
        if not solved:
            solved=isSolved(np_cells)

    logger.debug(np_cells)
    return solved

logger.debug('info')


In [152]:
class Row:
    def __init__(self, r, np_cells) -> None:
        self.row_index=r
        self.row=getRow(r,np_cells)
        logger.debug(self.row)
        self.np_cells=np_cells

        self.missings_map=self.getMissingsMap()
        
        logger.debug(self.missings_map)

    def getMissingValeusAndUnknownCells(self):
        missing_values={i for i in "123456789"}
        unknown_cells=[]
        for j in range(9):
            v=self.row[j]
            if v=='0':
                unknown_cells.append((self.row_index,j))
            else:
                missing_values.remove(v)
        logger.debug("unknown_cells: {}".format(unknown_cells) )
        logger.debug("missing_values".format(missing_values))
        return (missing_values, unknown_cells)

    def getMissingsMap(self):
        missing_values, unknown_cells = self.getMissingValeusAndUnknownCells()
        logger.debug("missing_values: {}, unknown_cells: {}".format(missing_values, unknown_cells))
        
        missings_map={}

        for key in missing_values:
            for cell in unknown_cells:
                logger.debug("key {}, cell: {}".format(key, cell))
                if self.couldfit(key, cell):
                    logger.debug("add {} to {}".format(cell, key))
                    addCellToMissingsMap(key, cell, missings_map)
                else:
                    logger.debug("could not add {} to {}".format(cell, key))
        return missings_map


    def couldfit(self, key, cell):
        row_index, col_index = cell[0],cell[1]

        blockList=getBlockList(row_index, col_index, self.np_cells)
       
        col=getCol(col_index, self.np_cells)
       
        result =True
        if key in blockList:
            logger.debug("found {} in {}".format(key, blockList))
            result=False
        else:
            logger.debug("No {} in row {}: {}".format(key, row_index, blockList))
        if key in col:
            logger.debug("found {} in {}".format(key, col))
            result=False
        else:
            logger.debug("No {} in col {}: {}".format(key,col_index, col))
       
        return result

    # def addpos(self, m, u):
    #     if m in self.missings.keys():
    #         poss=self.missings[m]
    #         poss.append(u)
    #     else:
    #         poss=[u]
    #         self.missings[m]=poss

    def removepos(self,removes):
        deleted=[]
        for r in removes:
            for key in self.missings_map.keys():
                if r in self.missings_map[key]:
                    self.missings_map[key].remove(r)
                    if len(self.missings_map[key])==0:
                        deleted.append(key)
                    
        for d in deleted:
            self.missings_map.pop(d)


    def calculate(self):
        removes=[]
        changed=False
        repeat=True
        while repeat:
            repeat=False
            for key in self.missings_map.keys():
                if len(self.missings_map[key]) == 1:
                    px, py=self.missings_map[key][0]
                    logger.debug("Solved {} at {}".format(key,self.missings_map[key][0]))
                    self.np_cells[px, py]=key
                    removes.append((px,py))
                    logger.debug(self.block)
                    changed=True
                    repeat=False
            self.removepos(removes)
        
        return changed
    
def calculate_Rows(np_cells):
    changed=False
    for row_index in range(3):
        logger.debug("solving row {}".format(row_index) )
        r=Row(row_index, np_cells)
        if r.calculate():
            changed=True
    return changed

In [153]:
import unittest

puzzle_str='070000043040009610800634900094052000358460020000800530080070091902100005007040802'
puzzle=getPuzzle(puzzle_str)
solution_str='679518243543729618821634957794352186358461729216897534485276391962183475137945862'
solution=getPuzzle(solution_str)

row=Row(0,puzzle)
class TestNotebook(unittest.TestCase):

    def test_getPuzzle(self):
        expected=np.array(
        [['0', '7', '0', '0', '0', '0', '0', '4', '3'],
        ['0', '4', '0', '0', '0', '9', '6', '1', '0'],
        ['8', '0', '0', '6', '3', '4', '9', '0', '0'],
        ['0', '9', '4', '0', '5', '2', '0', '0', '0'],
        ['3', '5', '8', '4', '6', '0', '0', '2', '0'],
        ['0', '0', '0', '8', '0', '0', '5', '3', '0'],
        ['0', '8', '0', '0', '7', '0', '0', '9', '1'],
        ['9', '0', '2', '1', '0', '0', '0', '0', '5'],
        ['0', '0', '7', '0', '4', '0', '8', '0', '2']])
        
        self.assertTrue(np.array_equal(puzzle,expected))

    def test_solved(self):
        self.assertEqual(isSolved(puzzle),False)

    def test_unsolved(self):
        self.assertEqual(isSolved(solution),True)

    def test_getBlockIndex(self):
        self.assertEqual(getBlockIndex(8,8),(2,2))

    def test_getBlock(self):
        expected=np.array(
            [['0','0','0'],
            ['0','2','0'],
            ['5','3','0']])
        result=getBlock(1,2,puzzle)
        self.assertTrue(np.array_equal(result,expected))

    def test_getBlockList(self):
        expected=np.array(['0','0','0','0','2','0','5','3','0'])
        result=getBlockList(1,2,puzzle)
        self.assertTrue(np.array_equal(result,expected))

    def test_getRow(self):
        expected=np.array(['0', '4', '0', '0', '0', '9', '6', '1', '0'])
        result=getRow(1,puzzle)
        self.assertTrue(np.array_equal(result,expected))

    def test_getCol(self):
        expected=np.array(['0','0','0','4','8','0','0','2','7'])
        
        result=getCol(2,puzzle)
        logger.debug(result)
        self.assertTrue(np.array_equal(result,expected))

    def test_getCol(self):
        expected=np.array(['0','0','0','4','8','0','0','2','7'])
        
        result=getCol(2,puzzle)
        logger.debug(result)
        self.assertTrue(np.array_equal(result,expected))

    def test_calculate_possibilitie(self):
        expected={'1','2','5','6'}
        
        result=calculate_possibilitie(0,0,puzzle)
        logger.debug(result)
        self.assertTrue(np.array_equal(result[2],expected))
    def test_Block(self):
        expected={'1','2','5','6','9','3'}
        block=Block(0,0,puzzle)
        keys=block.missings_map.keys()
        logger.debug("keys: {}".format(keys))
        # print("len: {}, result: {}".format(len(result),result))
        self.assertTrue(keys,expected)
    def test_Row(self):
        expected={'1','2','5','6','9','3'}
        row=Row(0,puzzle)
        keys=row.missings_map.keys()
        logger.debug("keys: {}".format(keys))
        # print("len: {}, result: {}".format(len(result),result))
        self.assertTrue(keys,expected)

unittest.main(argv=[''], verbosity=2, exit=False)

ValueError: cannot reshape array of size 0 into shape (9,)

In [50]:
df = pd.read_csv("./data/sudoku.csv")

In [74]:
np_cells=np.array(list(puzzle_str)).reshape(9,9)
np_cells

array([['0', '7', '0', '0', '0', '0', '0', '4', '3'],
       ['0', '4', '0', '0', '0', '9', '6', '1', '0'],
       ['8', '0', '0', '6', '3', '4', '9', '0', '0'],
       ['0', '9', '4', '0', '5', '2', '0', '0', '0'],
       ['3', '5', '8', '4', '6', '0', '0', '2', '0'],
       ['0', '0', '0', '8', '0', '0', '5', '3', '0'],
       ['0', '8', '0', '0', '7', '0', '0', '9', '1'],
       ['9', '0', '2', '1', '0', '0', '0', '0', '5'],
       ['0', '0', '7', '0', '4', '0', '8', '0', '2']], dtype='<U1')

In [140]:
np_cells=np.array(list(puzzle_str)).reshape(9,9)
single_solve(np_cells)
print(np_cells)

[['6' '7' '9' '5' '1' '8' '2' '4' '3']
 ['5' '4' '3' '7' '2' '9' '6' '1' '8']
 ['8' '2' '1' '6' '3' '4' '9' '5' '7']
 ['7' '9' '4' '3' '5' '2' '1' '8' '6']
 ['3' '5' '8' '4' '6' '1' '7' '2' '9']
 ['2' '1' '6' '8' '9' '7' '5' '3' '4']
 ['4' '8' '5' '2' '7' '6' '3' '9' '1']
 ['9' '6' '2' '1' '8' '3' '4' '7' '5']
 ['1' '3' '7' '9' '4' '5' '8' '6' '2']]


In [148]:
tests=df['puzzle'].loc[:100]
solved=0
for t in tests:
    np_cells=np.array(list(t)).reshape(9,9)
    if not single_solve(np_cells):
        print("Not solve: {}".format(np_cells))
    else:
        solved+=1

print("Solved: {}".format(solved))

Not solve: [['6' '2' '0' '7' '4' '5' '1' '0' '0']
 ['0' '7' '4' '1' '0' '0' '6' '5' '2']
 ['5' '1' '8' '2' '6' '9' '3' '7' '4']
 ['0' '6' '7' '3' '0' '0' '9' '0' '0']
 ['0' '9' '0' '0' '0' '0' '0' '6' '0']
 ['8' '4' '5' '9' '7' '6' '2' '3' '1']
 ['0' '8' '2' '0' '0' '0' '0' '0' '6']
 ['0' '3' '6' '8' '0' '0' '0' '2' '0']
 ['4' '5' '0' '6' '0' '2' '0' '0' '3']]
Not solve: [['2' '7' '8' '0' '0' '6' '1' '4' '5']
 ['9' '1' '3' '4' '0' '5' '7' '0' '0']
 ['4' '6' '5' '7' '0' '1' '9' '0' '0']
 ['5' '9' '2' '0' '7' '4' '8' '0' '1']
 ['8' '3' '1' '5' '0' '2' '4' '7' '0']
 ['6' '4' '7' '0' '1' '8' '5' '0' '0']
 ['1' '8' '4' '0' '0' '9' '3' '5' '7']
 ['7' '2' '9' '8' '5' '3' '6' '1' '4']
 ['3' '5' '6' '1' '4' '7' '2' '9' '8']]
Solved: 99
